# Baseline CRAB pulses

Side-by-side Bloch-sphere animation comparing two protocols derived from `data/baselines/_baseline_crab`: the drive-only pulse and the combined drive+detuning pulse.

In [ ]:
import json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
from qutip import mesolve, sigmax, sigmay, sigmaz, Bloch, Qobj

# Locate baseline irrespective of where the notebook is run

def find_baseline_dir():
    for root in [Path.cwd(), *Path.cwd().parents]:
        candidate = root / "data" / "baselines" / "_baseline_crab"
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Could not find data/baselines/_baseline_crab from this working directory")

BASELINE_DIR = find_baseline_dir()
print(f"Using baseline at {BASELINE_DIR}")

with open(BASELINE_DIR / "metadata.json") as f:
    metadata = json.load(f)
arrays = np.load(BASELINE_DIR / "arrays.npz")

t = arrays["t"]                     # microseconds
omega = arrays["Omega0"]            # rad/us drive
delta = arrays["Delta0"]            # rad/us detuning
rho0 = Qobj(arrays["rho0"])        # initial state

print(f"Loaded baseline: Nt={metadata['summary']['Nt']}, T={metadata['summary']['T_us']} us")


In [ ]:
def simulate_protocol(omega_t, delta_t, label):
    '''Propagate the qubit under the provided controls and return Bloch coordinates.'''
    H = [
        [0.5 * sigmax(), lambda tt, args: np.interp(tt, t, omega_t)],
        [0.5 * sigmaz(), lambda tt, args: np.interp(tt, t, delta_t)],
    ]
    result = mesolve(H, rho0, t, e_ops=[sigmax(), sigmay(), sigmaz()])
    coords = np.vstack(result.expect)  # shape (3, len(t))
    return {"label": label, "coords": coords}

protocols = [
    simulate_protocol(omega, np.zeros_like(delta), "Pulse 1: Ω only"),
    simulate_protocol(omega, delta, "Pulse 2: Ω + Δ"),
]

# Downsample for smoother animations if needed
step = max(1, len(t) // 400)
for proto in protocols:
    proto["coords"] = proto["coords"][:, ::step]

print("Protocols simulated. Ready to animate.")


In [ ]:
def bloch_animation(protocols, interval_ms=40):
    fig = plt.figure(figsize=(12, 6))
    axes = [fig.add_subplot(1, 2, i + 1, projection="3d") for i in range(len(protocols))]

    bloch_objs = []
    for ax, proto in zip(axes, protocols):
        b = Bloch(fig=fig, axes=ax)
        b.frame_alpha = 0.08
        b.sphere_alpha = 0.09
        b.point_color = ["#d62728"]
        b.point_marker = ["o"]
        b.vector_color = ["#1f77b4"]
        b.title = proto["label"]
        bloch_objs.append(b)

    n_frames = protocols[0]["coords"].shape[1]

    def draw(frame):
        for b, proto in zip(bloch_objs, protocols):
            coords = proto["coords"]
            b.clear()
            b.add_points([coords[0, : frame + 1], coords[1, : frame + 1], coords[2, : frame + 1]], meth="l")
            b.add_points([coords[0, frame], coords[1, frame], coords[2, frame]], meth="p")
            b.make_sphere()
            b.axes.set_title(proto["label"])
        return fig,

    anim = animation.FuncAnimation(fig, draw, frames=n_frames, interval=interval_ms, blit=False)
    plt.close(fig)
    return anim

anim = bloch_animation(protocols)
display(HTML(anim.to_jshtml()))
